In [1]:
import os
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import glob
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import timm
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from scipy.stats import spearmanr
from PIL import Image
from sklearn.preprocessing import MinMaxScaler

In [2]:
DATA_STRUCTURE_PATH = "/kaggle/input/muen-structure/download_checkpoint_muen_old.json"

In [3]:
def set_seed():
    seed = 28
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True    
set_seed()

In [4]:
class EfficientNetRegressor(nn.Module):
    def __init__(self, model_name='efficientnet_b3'):
        super(EfficientNetRegressor, self).__init__()
        
        self.backbone = timm.create_model(model_name, 
                                         pretrained=True,
                                         num_classes=0,
                                         global_pool='avg')
        
        # Single output neuron for regression
        self.regressor = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.backbone.num_features, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),
            
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),
            
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),
            
            nn.Dropout(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        features = self.backbone(x)
        return self.regressor(features)

class StreetViewDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label']
        
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

def transform_normalized_to_euros_single(normalized_value, scaler):
    min_log = scaler.data_min_[0]
    max_log = scaler.data_max_[0]
    log_value = normalized_value * (max_log - min_log) + min_log
    rent_price = np.expm1(log_value)
    return rent_price

In [7]:
# Usage:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

munich_model = EfficientNetRegressor(model_name='efficientnet_b3')  
munich_model.load_state_dict(torch.load('/kaggle/input/best-regression-model-muenchen/best_regression_model_muenchen.pth', map_location=torch.device('cpu')))
munich_model.to(device)

Using device: cpu


EfficientNetRegressor(
  (backbone): EfficientNet(
    (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNormAct2d(
            40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          

In [36]:
# Transform
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load image
image_path = "/kaggle/input/cam-muenchen-vortrag/München_vortrag/Schwantlerhöhe/48.14062727876648,11.539384630902731.jpg"
image = Image.open(image_path).convert('RGB')

# transformation + batch dim
image_tensor = transform(image).unsqueeze(0)

image_tensor = image_tensor.to(device)

# Set model to evaluation mode and make prediction
munich_model.eval()
with torch.no_grad():
    output = munich_model(image_tensor)
    prediction = output.item()  # Convert to Python float

print(f"Normalized price {prediction:.2f}")

Normalized price 0.74


In [37]:
with open(DATA_STRUCTURE_PATH, 'r') as f:
    data = json.load(f)

rent_prices = {}
districts = []

for item in data['successful_downloads']:
    district = item['district']
    districts.append(district)
    rent_price = item['rent_price']
    rent_prices[district] = rent_price

rent_prices
districts = list(set(districts))

rent = []
for district in districts:
    rent.append(rent_prices[district])

log_rent = np.log1p(rent)
log_rent_2d = log_rent.reshape(-1, 1)
scaler = scaler = MinMaxScaler()
normalized_rent = scaler.fit_transform(log_rent_2d)

min_log = scaler.data_min_[0]
max_log = scaler.data_max_[0]

print(f'{transform_normalized_to_euros_single(prediction, scaler):.2f} Euro')

25.75 Euro


In [38]:
pred_value = transform_normalized_to_euros_single(prediction, scaler)
true_label = 25.35

print(f"True label: {true_label}")
print(f"Predicted: {pred_value:.2f} Euro")
print(f"Error: {abs(pred_value - true_label):.2f}\n")

True label: 25.35
Predicted: 25.75 Euro
Error: 0.40

